## Post-process a finetuned LLM

Test and upload a finetuned language model

In [1]:
!pip install -q -U huggingface_hub peft transformers torch accelerate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!git config --global credential.helper store

## Setup

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [2]:
!nvidia-smi

Mon Jul 24 20:43:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 PCIe    On   | 00000000:06:00.0 Off |                    0 |
| N/A   38C    P0    52W / 350W |      3MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"
n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
print("max memory: ", max_memory)

max memory:  {0: '76GB'}


## Loss curve

During training, the model converged nicely as follows:

![image](https://raw.githubusercontent.com/daniel-furman/sft-demos/main/assets/jul_24_23_1_14_00_log_loss_curves_llama-2-70b-dolphin.png)


## Basic testing

With a supervised finetuned (sft) model in hand, we can test it on some basic prompts and then upload it to the Hugging Face hub either as a public or private model repo, depending on the use case.

In [5]:
# peft_model_id = "results/checkpoint-12500"
peft_model_id = "dfurman/llama-2-70b-dolphin-peft"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    use_auth_token=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 8192, padding_idx=0)
        (layers): ModuleList(
          (0-79): 80 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=8192, out_features=8192, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=8192, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=8192, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=8192, out_features=1024, bias=False

In [7]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"
n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
print("max memory: ", max_memory)

max memory:  {0: '37GB'}


In [8]:
# text generation function


def llama_generate(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    prompt: str,
    max_new_tokens: int = 128,
    temperature: int = 1.0,
) -> str:
    """
    Initialize the pipeline
    Uses Hugging Face GenerationConfig defaults
        https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig
    Args:
        model (transformers.AutoModelForCausalLM): Falcon model for text generation
        tokenizer (transformers.AutoTokenizer): Tokenizer for model
        prompt (str): Prompt for text generation
        max_new_tokens (int, optional): Max new tokens after the prompt to generate. Defaults to 128.
        temperature (float, optional): The value used to modulate the next token probabilities.
            Defaults to 1.0
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inputs = tokenizer(
        [prompt],
        return_tensors="pt",
        return_token_type_ids=False,
    ).to(
        device
    )  # tokenize inputs, load on device

    # when running Torch modules in lower precision, it is best practice to use the torch.autocast context manager.
    with torch.autocast("cuda", dtype=torch.bfloat16):
        response = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            return_dict_in_generate=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded_output = tokenizer.decode(
        response["sequences"][0],
        skip_special_tokens=True,
    )  # grab output in natural language

    return decoded_output[len(prompt) :]  # remove prompt from output

In [9]:
prompt = "You are a helpful assistant. Write me a numbered list of things to do in New York City.\n"

response = llama_generate(
    model,
    tokenizer,
    prompt,
    max_new_tokens=250,
    temperature=0.92,
)

print(response)

1. Visit the Statue of Liberty: Take a ferry to Liberty Island and see the iconic symbol of freedom and democracy.
2. Explore Central Park: Enjoy the lush greenery, take a stroll, or rent a bike to explore the park's many attractions, such as the Central Park Zoo and Strawberry Fields.
3. Visit the Empire State Building: Take an elevator to the observation deck for breathtaking views of the city.
4. Walk the Brooklyn Bridge: Cross the historic bridge and enjoy the stunning views of the Manhattan skyline.
5. Experience Times Square: Take in the bright lights, billboards, and bustling energy of this iconic intersection.
6. Visit the Metropolitan Museum of Art: Explore the vast collection of art and artifacts from around the world.
7. Take a food tour: Sample the diverse cuisine of New York City, from pizza to bagels to ethnic specialties.
8. Attend a Broadway show: Enjoy a world-class performance in the heart of the Theater District.
9


In [31]:
prompt = "You are a helpful assistant. Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense? Respond only with a yes or no answer in as few words as possible.\n"

response = llama_generate(
    model,
    tokenizer,
    prompt,
    max_new_tokens=100,
    temperature=0.92,
)

print(response)

No.

Explanation: Daniel's barber is not available on Sundays, so he cannot get a haircut on that day. He should go on a day when the barber is working, such as Monday, Wednesday, or Friday.

So, the answer is no, it does not make logical sense for Daniel to go for a haircut on Sunday. He should go on a day when the barber is available.

In as


In [11]:
prompt = "You are a helpful assistant. Write a short email inviting my friends to a dinner party on Friday. Respond succinctly.\n"

response = llama_generate(
    model,
    tokenizer,
    prompt,
    max_new_tokens=200,
    temperature=0.92,
)

print(response)

Subject: Dinner Party Invitation - Friday, 7 PM

Dear Friends,

I hope this email finds you well! I'm excited to invite you to a dinner party at my place this Friday, at 7 PM. We'll have a delicious meal, great company, and lots of fun. Please let me know if you can make it, so I can plan accordingly.

Looking forward to seeing you all!

Kind regards,
Your Name

P.S. Please let me know if you have any dietary restrictions or allergies.










































































In [19]:
prompt = "You are a helpful assistant. Tell me a recipe for vegan banana bread.\n"

response = llama_generate(
    model,
    tokenizer,
    prompt,
    max_new_tokens=500,
    temperature=0.92,
)

print(response)

Sure! Here's a delicious and easy vegan banana bread recipe:

Ingredients:
- 2 cups all-purpose flour
- 1/2 cup sugar
- 1/2 cup vegan butter (such as Earth Balance)
- 1/2 cup vegan milk (such as almond milk)
- 1/2 cup unsweetened applesauce
- 1/2 cup mashed ripe bananas (about 2 medium bananas)
- 1 teaspoon baking soda
- 1/2 teaspoon salt
- 1/2 teaspoon ground cinnamon
- 1/2 teaspoon ground nutmeg
- 1/2 teaspoon ground cloves
- 1/2 cup chopped walnuts (optional)

Instructions:
1. Preheat the oven to 350°F (175°C). Grease a 9x5-inch loaf pan with vegan butter or cooking spray.
2. In a large bowl, mix together the flour, sugar, vegan butter, vegan milk, applesauce, bananas, baking soda, salt, cinnamon, nutmeg, and cloves. Stir until well combined.
3. Fold in the chopped walnuts, if using.
4. Pour the batter into the prepared loaf pan.
5. Bake for 50-60 minutes, or until a toothpick inserted into the center of the bread comes out clean.
6. Let the bread cool in the pan for 10 minutes befo

## Inf runtime test

In [13]:
import tqdm
import time

prompt = "You are a helpful assistant. Write me a long list of things to do in San Francisco:\n"

runtimes = []
for i in tqdm.tqdm(range(25)):
    start = time.time()
    response = llama_generate(
        model,
        tokenizer,
        prompt,
        max_new_tokens=50,
        temperature=0.92,
    )
    end = time.time()
    runtimes.append(end - start)
    assert len(tokenizer.encode(response)) == 52

100%|██████████| 25/25 [01:52<00:00,  4.49s/it]


In [14]:
avg_runtime = torch.mean(torch.tensor(runtimes)).item()
print(f"Runtime avg in seconds: {avg_runtime}")  # time in seconds

Runtime avg in seconds: 4.487184524536133


## Upload model to Hugging Face
1. Before running the cells below, create a model on your Hugging Face account. It can be a private or public repo and work with the below code.

In [10]:
# push to hub
model_id_load = "dfurman/llama-2-70b-dolphin-peft"

# tokenizer
tokenizer.push_to_hub(model_id_load, use_auth_token=True)
# safetensors
model.push_to_hub(model_id_load, use_auth_token=True, safe_serialization=True)
# torch tensors
model.push_to_hub(model_id_load, use_auth_token=True)

adapter_model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dfurman/llama-2-70b-dolphin-peft/commit/c2f027b2944309176e6c666785371c61c1e13c1c', commit_message='Upload model', commit_description='', oid='c2f027b2944309176e6c666785371c61c1e13c1c', pr_url=None, pr_revision=None, pr_num=None)